In [13]:
from sqlalchemy import create_engine, Integer, insert
from sqlalchemy.schema import Table, MetaData, Column
from sqlalchemy.sql.expression import select, text
from sqlalchemy.exc import SQLAlchemyError, DBAPIError
import pandas as pd

In [ ]:
!./starrocks_docker.sh

In [14]:
engine = create_engine('starrocks://root:@localhost:9030/product_recommendation')

In [ ]:
def read_file_to_string(filepath):
  try:
    with open(filepath, 'r') as file:
      file_content = file.read()
    return file_content
  except FileNotFoundError:
    print(f"Error: File not found at path: {filepath}")
    return None
  except Exception as e:
    print(f"An error occurred: {e}")
    return None


In [15]:
def sql_execute(engine, sql):
    try:
        with engine.connect() as connection:
            result = connection.execute(text(sql))
            rows = result.fetchall()
            return engine, rows
    except (SQLAlchemyError, DBAPIError) as e:
        print(f"A database error occurred: {e}")
        # Perform any necessary cleanup here, such as rolling back transactions.
        # For example: connection.rollback()  (if you're in a transaction)

        # Optionally, you can re-raise a specific exception or a custom exception
        # to provide more context to the caller.  For instance:
        # raise DatabaseError("Failed to execute SQL query", e) from e
        raise  # Re-raise the original exception
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise  # Re-raise the exception

In [16]:
sql_query = "SELECT 1"

try:
    engine, rows = sql_execute(engine, sql_query)
    print("Query executed successfully!")
    print("Rows:", rows)
except SQLAlchemyError:
    print("The calling function is handling the database error.")
except Exception:
    print("The calling function is handling unexpected error.")


sql_query_error = "SELECT invalid_column FROM non_existent_table"
try:
    engine, rows = sql_execute(engine, sql_query_error)
    print("Query executed successfully!")
    print("Rows:", rows)
except SQLAlchemyError:
    print("The calling function is handling the database error.")
except Exception:
    print("The calling function is handling unexpected error.")

Query executed successfully!
Rows: [(1,)]
A database error occurred: (pymysql.err.OperationalError) (5502, "Getting analyzing error. Detail message: Unknown table 'product_recommendation.non_existent_table'.")
[SQL: SELECT invalid_column FROM non_existent_table]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
The calling function is handling the database error.


In [ ]:
engine = create_engine('starrocks://root:@localhost:9030/product_recommendation')
filepath = "initiate_db.sql"
initiate_db = read_file_to_string(filepath)
engine,rows = sql_execute(engine,initiate_db)

In [ ]:
!./stream_load_starrocks.sh